## Pre-Process Data ##

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import sklearn as sk
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split
import os
from LinearRegression import LinearRegression
from LogisticRegression import CustomLogisticRegression
from MultiClassRegression import MultiClassRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression as LR
from collections import defaultdict
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_curve, auc
from scipy.stats import gaussian_kde 
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder

### Extact, Transform, Load (ETL) ###

In [ ]:

def load_vocab(vocab_file_path):
    """
    Loads the vocabulary from a file and creates two dictionaries: 
    one mapping indices to words and another mapping words to indices.
    """
    index_to_feature_map = {}
    with open(vocab_file_path, 'r', encoding='utf-8') as vocab_file:
        for index, word in enumerate(vocab_file):
            index_to_feature_map[str(index)] = word.strip()
    feature_to_index_map = {word: index for index, word in index_to_feature_map.items()}
    return index_to_feature_map, feature_to_index_map

def parse_review_features(line):
    features = line.strip().split()[1:]  # Skip the first element (label)
    return [feature.split(':') for feature in features]

def compute_review_frequencies(feature_path):
    """
    Computes the frequency of each word across reviews.
    """
    review_frequencies = defaultdict(int)
    total_reviews = 0
    with open(feature_path, 'r', encoding='utf-8') as feat_file:
        for line in feat_file:
            total_reviews += 1
            for index, _ in parse_review_features(line):
                review_frequencies[index] += 1
    return review_frequencies, total_reviews

def filter_feature_indices(review_frequencies, total_reviews, lower_bound=0.01, upper_bound=0.5):
    """
    Filters out feature indices based on review frequency criteria.
    """
    min_reviews = total_reviews * lower_bound
    max_reviews = total_reviews * upper_bound
    filtered_feature_indices = {index for index, freq in review_frequencies.items() 
                                if min_reviews <= freq <= max_reviews}

    return sorted([int(i) for i in filtered_feature_indices])

def load_review_features(feature_path, index_mapping, n_reviews, n_features):
    """
    Loads review features and labels into matrices based on a filtered and remapped vocabulary.
    """
    X = np.zeros((n_reviews, n_features))
    y = np.zeros(n_reviews)
    with open(feature_path, 'r') as file:
        for review_id, line in enumerate(file):
            parts = line.strip().split()
            y[review_id] = int(parts[0])  # First part is the label
            for index, count in parse_review_features(line):
                if index in index_mapping:
                    X[review_id, index_mapping[index]] = int(count)
    return X, y

def generate_frequency_table(vocab_file_path, feature_path):
    """
    Generates a frequency table of words in the dataset.
    
    Args:
    - vocab_file_path: Path to the vocabulary file.
    - feature_path: Path to the feature file containing word occurrences in reviews.
    
    Returns:
    - A sorted list of tuples, where each tuple contains (word, frequency).
    """
    index_to_word, word_to_index = load_vocab(vocab_file_path)
    review_frequencies, total_reviews = compute_review_frequencies(feature_path)
    word_frequencies = defaultdict(int)
    for index, freq in review_frequencies.items():
        word = index_to_word[index]
        word_frequencies[word] = freq
        
    sorted_word_frequencies = sorted(word_frequencies.items(), key=lambda x: x[1], reverse=True)
    return sorted_word_frequencies


def filter_by_regression_coefficients(X, y, D=1000):
    model = LinearRegression()
    model.fit(X, y)
    coefficients = model.w[:-1]
    top_indices = np.argsort(np.abs(coefficients))[-D:]
    return top_indices, coefficients

def filter_by_logistic_regression_coefficients(X, y, D=1000):
    model = CustomLogisticRegression(learning_rate=0.01, epsilon=1e-5, max_iters=1e4)
    model.fit(X, y)
    coefficients = model.w[:-1]
    top_indices = np.argsort(np.abs(coefficients))[-D:]
    return top_indices, coefficients

# Evaluation and Plotting Functions #

### Evaluation Functions ###

In [ ]:
  
def evaluate_multiclass_classification():
    """
    Compute classification accuracy.
    """
    #Load the dataset
    categories = ['comp.graphics', 'misc.forsale', 'rec.sport.baseball', 'sci.med', 'talk.politics.guns']
    train = fetch_20newsgroups(subset='train', categories=categories, remove=('headers', 'footers', 'quotes'))
    test = fetch_20newsgroups(subset='test', categories=categories, remove=('headers', 'footers', 'quotes'))

    #Convert text to feature vectors
    count_vect = CountVectorizer(stop_words='english', max_df=0.5, min_df=2, ngram_range=(1, 2))
    X_train_counts = count_vect.fit_transform(train.data)
    tfidf_transformer = TfidfTransformer()
    X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

    #Feature selection using Mutual Information
    M = 500  # Select top M features
    feature_selection = SelectKBest(mutual_info_classif, k=M)
    X_train_selected = feature_selection.fit_transform(X_train_tfidf, train.target)
    X_test_counts = count_vect.transform(test.data)
    X_test_tfidf = tfidf_transformer.transform(X_test_counts)
    X_test_selected = feature_selection.transform(X_test_tfidf)

    #Train the MultiClassRegression model
    n_features = X_train_selected.shape[1]
    n_classes = len(categories)
    mc_model = MultiClassRegression(nFeatures=n_features, nClasses=n_classes)
    mc_model.fit(X_train_selected.toarray(), train.target, lr=0.05, niters=2000, verbose=False)

    #Evaluate model
    y_pred = mc_model.predict(X_test_selected.toarray())
    accuracy = accuracy_score(test.target, y_pred)
    print(f"Test Accuracy: {accuracy}")


evaluate_multiclass_classification()

### Plotting Functions ###

In [ ]:



def plot_top_features(coefficients, sorted_indices, index_to_feature_map, reverse_index_mapping, title, n_features=10):
    """
    Plots the top N negative and positive features based on their coefficients, with a legend.
    
    Parameters:
    - coefficients: Array of feature coefficients from a regression model.
    - index_to_feature_map: Dictionary mapping original feature indices to words.
    - reverse_index_mapping: Dictionary mapping from reduced feature space to original index.
    - n_features: Number of top features to plot for both negative and positive coefficients.
    """
    
    # Separate positive and negative coefficients
    positive_indices = [i for i, coef in enumerate(coefficients) if coef > 0]
    negative_indices = [i for i, coef in enumerate(coefficients) if coef < 0]
    
    # Sort them by absolute values but keep the sign for color coding
    top_positive_indices = sorted(positive_indices, key=lambda i: coefficients[i], reverse=True)[:n_features]
    top_negative_indices = sorted(negative_indices, key=lambda i: coefficients[i])[:n_features]
    
    # Combine the indices and coefficients for plotting
    top_indices = top_negative_indices + top_positive_indices
    top_coefficients = [coefficients[i] for i in top_indices]
    top_words = [index_to_feature_map[str(reverse_index_mapping[i])] for i in top_indices]
    
    # Determine colors based on coefficient sign
    colors = ['red' if coef < 0 else 'blue' for coef in top_coefficients]
    
    # Plot
    plt.figure(figsize=(10, 8))
    sns.barplot(x=np.abs(top_coefficients), y=top_words, palette=colors, orient='h')
    plt.xlabel('Coefficient Value')
    plt.ylabel('Features')
    plt.title(title)
    
    # Create a custom legend
    from matplotlib.patches import Patch
    legend_elements = [Patch(facecolor='blue', edgecolor='blue', label='Positive'),
                       Patch(facecolor='red', edgecolor='red', label='Negative')]
    plt.legend(handles=legend_elements, title='Coefficient Sign')
    plt.savefig(f'experiment_1_results_{title}.png')
    plt.show()

def plot_training_progress(logistic_regression_model):
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(logistic_regression_model.loss_history, label='Loss', color='b', ls='-',linewidth=1)
    plt.title('Loss Progress over Iterations')
    plt.xlabel('Iteration')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(logistic_regression_model.gradient_norm_history, label='Gradient Norm')
    plt.title('Gradient Norm Progress over Iterations')
    plt.xlabel('Iteration')
    plt.ylabel('Gradient Norm')
    plt.legend()
    plt.tight_layout()
    plt.savefig('experiment_2_training_progress_results.png')
    plt.show()
    
def plot_model_convergence(histories, results):
    """
    Display plots for model convergence: ROC curves, training losses, and gradient norms
    for different learning rates.
    
    Args:
    - histories: Dictionary mapping learning rates (str) to tuples of (loss_history, gradient_norm_history).
    - results: Dictionary mapping learning rates (str) to AUROC scores.
    """
    plt.figure(figsize=(18, 6))

    # Plot ROC curves
    plot_multiple_roc_curves(results, 'experiment_7_roc_results.png')

    # Plot training losses
    plt.subplot(1, 3, 2)
    for lr, (loss_history, _) in histories.items():
        plt.plot(loss_history, label=f'LR={lr}')
    plt.title('Training Loss Progress')
    plt.xlabel('Iteration')
    plt.ylabel('Loss')
    plt.legend()

    # Plot gradient norms
    plt.subplot(1, 3, 3)
    for lr, (_, gradient_norm_history) in histories.items():
        plt.plot(gradient_norm_history, label=f'LR={lr}')
    plt.title('Gradient Norm Progress')
    plt.xlabel('Iteration')
    plt.ylabel('Gradient Norm')
    plt.legend()

    plt.tight_layout()
    plt.savefig('experiment_7_training_results.png')
    plt.show()

def plot_binary_classification(results):
    """
    plot ROC curve
    compare to DT from sklearn
    
    Args:
    - results: tuple of fpr, tpr, auroc
    
    Return:
    - None, plot the ROC curve
    
    """
    fpr, tpr, auroc = results
    print(f"Area under the ROC curve: {auroc}")
    print(f"True positive rate: {tpr}")
    print(f"False positive rate: {fpr}")
    plt.figure()
    plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % auroc)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc="lower right")
    plt.savefig('experiment_4_results.png')
    plt.show()
    
def plot_multiple_roc_curves(results_dict,title):
    """
    Plot ROC curves for multiple classifiers.
    
    Args:
    - results_dict: Dictionary mapping model names (str) to tuples of (fpr, tpr, auroc).
    
    Return:
    - None, plot the ROC curves.
    """
    plt.figure(figsize=(10, 8))

    # Iterate through the results dictionary and plot each ROC curve
    for model_name, (fpr, tpr, auroc) in results_dict.items():
        plt.plot(fpr, tpr, lw=2, label=f'{model_name} ROC curve (area = {auroc:.2f})')
    
    # Plot diagonal line for no skill classifier
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc="lower right")
    plt.savefig(title)
    plt.show()
    

def plot_multiclass_classification(results):
    """
    compare accuracy to DT from sklearn
    
    """
    pass

def plot_top_20_features_from_imdb_simple(top_positive_features, top_negative_features):
    """
    A horizontal bar plot showing the top 20 features from the Simple linear regression on the IMDB data
    
    Characteristics:
    - 10 most positive and negative coefficients as the x-axis 
    - Feature names (i.e., words) as the y-axis  
    """
    # Plot the top 20 positive and negative features on the same plot
    plt.figure(figsize=(10, 10))
    # assign each a different color and make negatives positive
    top_negative_features['Impact'] = top_negative_features['Impact'].abs()
    plt.barh(top_positive_features['Word'].head(10), top_positive_features['Impact'].head(10), color='b', label='Positive Impact')
    plt.barh(top_negative_features['Word'].head(10), top_negative_features['Impact'].head(10), color='r', label='Negative Impact')
    plt.xlabel('Regression Coefficients (Absolute Value)')
    plt.ylabel('Word')
    plt.title('Top 20 Features from Simple Linear Regression')
    plt.legend()
    plt.savefig('top_20_features_from_imdb_simple.png')
    plt.show()


def plot_model_convergence(histories, results):
    """
    Display plots for model convergence: ROC curves, training losses, and gradient norms
    for different learning rates.
    
    Args:
    - histories: Dictionary mapping learning rates (str) to tuples of (loss_history, gradient_norm_history).
    - results: Dictionary mapping learning rates (str) to AUROC scores.
    """
    plt.figure(figsize=(18, 6))

    # Plot training losses
    plt.subplot(1, 2, 1)
    for lr, (loss_history, _) in histories.items():
        plt.plot(loss_history, label=f'LR={lr}')
    plt.title('Training Loss Progress')
    plt.xlabel('Iteration')
    plt.ylabel('Loss')
    plt.legend()

    # Plot gradient norms
    plt.subplot(1, 2, 2)
    for lr, (_, gradient_norm_history) in histories.items():
        plt.plot(gradient_norm_history, label=f'LR={lr}')
    plt.title('Gradient Norm Progress')
    plt.xlabel('Iteration')
    plt.ylabel('Gradient Norm')
    plt.legend()
    
    plt.savefig('experiment_6_results.png')
    plt.tight_layout()
    plt.show()

def plot_imdb_data_auroc(results):
    """
    A bar plot that shows the AUROC of logistic regression and DT on the test data (y-axis) 
    as a function of the 20%, 40%, 60%, 80%, and 100% training data (x-axis).
    """
    # Extracting percentages and scores
    percentages = list(results.keys())
    auroc_logs = [result[0] for result in results.values()]
    auroc_dts = [result[1] for result in results.values()]
    
    # Setting up the bar plot with an adjusted figure size
    plt.figure(figsize=(12, 8))  # Adjust figure size as needed
    ax = plt.subplot(111)
    index = np.arange(len(results))
    bar_width = 0.35
    opacity = 0.8
    
    bars1 = ax.bar(index, auroc_logs, bar_width, alpha=opacity, color='b', label='Logistic Regression')
    bars2 = ax.bar(index + bar_width, auroc_dts, bar_width, alpha=opacity, color='g', label='Decision Tree')
    
    # Adding labels, title, and axes ticks
    ax.set_xlabel('Training Set Size (%)')
    ax.set_ylabel('AUROC')
    ax.set_title('AUROC by Model and Training Set Size')
    ax.set_xticks(index + bar_width / 2)
    ax.set_xticklabels(percentages)
    
    # Adjusting the legend position to avoid overlap and ensure it's inside the plot area
    ax.legend(loc='upper left', bbox_to_anchor=(1.05, 1), borderaxespad=0.)
    
    # Add labels onto the bars with some adjustments for better readability
    for bars in [bars1, bars2]:
        for bar in bars:
            height = bar.get_height()
            ax.annotate(f'{height:.2f}',
                        xy=(bar.get_x() + bar.get_width() / 2, height),
                        xytext=(0, 3),  # 3 points vertical offset
                        textcoords="offset points",
                        ha='center', va='bottom')
    
    # Adjust layout to make room for the legend and ensure labels are not cut off
    plt.subplots_adjust(right=0.75)
    plt.tight_layout()
    plt.savefig('experiment_5_results.png')
    plt.show()
    
def plot_word_distribution(review_frequencies, title):
    """
    Plots a histogram and distribution of word counts in a dataset.
    Args:
    - data: Tuple containing (review_frequencies, total_reviews), where review_frequencies
      is a dictionary mapping word indices to frequencies, and total_reviews is the total number of reviews.
    - title: Title for the plot.
    """
    indices = np.array([int(index) for index in review_frequencies.keys()])
    frequencies = np.array(list(review_frequencies.values()))
    
    plt.figure(figsize=(14, 6))
    
    plt.subplot(1, 2, 1)
    plt.hist(indices, weights=frequencies, bins=100, color='skyblue', edgecolor='black')
    plt.title(f'{title} - Histogram')
    plt.xlabel('Word Index')
    plt.ylabel('Frequency')
    plt.yscale('log', nonpositive='clip')
    plt.grid(True, which="both", ls="--")
    
    plt.subplot(1, 2, 2)
    kde = gaussian_kde(indices, weights=frequencies)
    idx_range = np.linspace(indices.min(), indices.max(), 300)
    plt.plot(idx_range, kde(idx_range), color='darkblue', lw=2)
    plt.title(f'{title} - Density Plot')
    plt.xlabel('Word Index')
    plt.ylabel('Density')
    plt.yscale('log', nonpositive='clip')

    plt.tight_layout()
    plt.savefig(f'imdb_dataset_word_distribution.png')
    plt.show()
    
    
def plot_news_data_classification_accuracy(results):
    """
    A bar plot that shows the classification accuracies of multiclass regression and DT 
    on the test data (y-axis) as a function of the 20%, 40%, 60%, 80%, and 100% training data (x- axis)
    """
    pass

def plot_top_20_features_from_imdb_logistic(results):
    """
    A horizontal bar plot showing the top 20 features (10 most positive and 10 most negative) 
    from the logistic regression on the IMDB data with the coefficient as the x-axis and the feature names (i.e., words) as the y-axis
    """
    pass

def plot_heatmap_of_multi_classification(results):
    """
    A heatmap showing the top 5 most positive features as rows for each class as columns in the multi-class classification 
    on 4 the chosen classes from the 20-news group datasets. Therefore, your heatmap should be a 20-by-4 dimension.

    """
    pass

# Experiments #

In [ ]:

# Driver code to load everything

# Define globals
VOCAB_FILE = '../aclImdb/imdb.vocab'
TRAIN_FEAT = '../aclImdb/train/labeledBow.feat'
TEST_FEAT = '../aclImdb/test/labeledBow.feat'
N_REVIEWS = 25000

# load the data and filter the reviews
index_to_feature_map, feature_to_index_map = load_vocab(VOCAB_FILE)
review_frequencies, total_reviews = compute_review_frequencies(TRAIN_FEAT)
filtered_indices = filter_feature_indices(review_frequencies, total_reviews)
index_mapping = {str(old_index): new_index for new_index, old_index in enumerate(filtered_indices)}

# plot the distribution
plot_word_distribution(review_frequencies, "Distribution of Words in the IMDB Dataset")

# create train and test based on the filtered indices
N_FEATURES = len(filtered_indices)
X_train, y_train = load_review_features(TRAIN_FEAT, index_mapping, N_REVIEWS, N_FEATURES)
X_test, y_test = load_review_features(TEST_FEAT, index_mapping, N_REVIEWS, N_FEATURES)

# Now filter by the regression coefficients
top_indices, coefficients = filter_by_regression_coefficients(X_train, y_train)
reverse_index_mapping = {new_index: old_index for old_index, new_index in index_mapping.items()}

# Convert y_train and y_test to binary
y_train_binary = (y_train > 5).astype(int)
y_test_binary = (y_test > 5).astype(int)

# Filter by regression coefficients from logistic regression
#top_log_indices, top_log_coefficients = filter_by_logistic_regression_coefficients(X_train, y_train_binary)
#reverse_index_mapping = {new_index: old_index for old_index, new_index in index_mapping.items()}

# Filter features based on top_indices
X_train_filtered = X_train[:, top_indices]
X_test_filtered = X_test[:, top_indices]

### Experiment 1 ###

In [ ]:
def experiment_one():
    """
    Report the:
    - top 10 features with the most positive coefficients
    - top 10 features with the most negative coefficients 
    
    on the IMDB data using simple linear regression on the movie rating scores
    """
    title1 = 'Top 20 Features by Absolute Coefficient Value From Linear Regression'
    title2 = 'Top 20 Features by Absolute Coefficient Value From Logistic Regression'
    plot_top_features(coefficients, top_indices[::-1], index_to_feature_map, reverse_index_mapping, title1, n_features=10)
    plot_top_features(top_log_coefficients, top_log_indices[::-1], index_to_feature_map, reverse_index_mapping, title2, n_features=10)
    

#experiment_one()

### Experiment 2 ###

In [ ]:
def experiment_two(X_train, y_train, X_test, y_test, top_indices):
    """
    Implement and conduct:
    - Binary classification on the IMDb Reviews
    - Multi-class classification on the 20 news group dataset
    """
    
    model = CustomLogisticRegression(learning_rate=0.01, epsilon=1e-5, max_iters=1e4,record_training=True)
    model.fit(X_train_filtered, y_train_binary)
    plot_training_progress(model)
    y_pred_prob_test = model.predict(X_test_filtered)
    y_pred_prob_train = model.predict(X_train_filtered)
    
    # Compute ROC curve and ROC area for the test set
    fpr_test, tpr_test, _ = roc_curve(y_test_binary, y_pred_prob_test)
    roc_auc_test = auc(fpr_test, tpr_test)
    
    # Compute ROC curve and ROC area for the training set
    fpr_train, tpr_train, _ = roc_curve(y_train_binary, y_pred_prob_train)
    roc_auc_train = auc(fpr_train, tpr_train)
    results = {"Logisitic Regression (Train)": (fpr_train, tpr_train, roc_auc_train), 
               "Logisitic Regression (Test)": (fpr_test, tpr_test, roc_auc_test)}
    plot_multiple_roc_curves(results, 'experiment_2_results.png')
    
#experiment_two(X_train_filtered, y_train_binary, X_test_filtered, y_test_binary, top_indices)

### Experiment 3 ###

In [ ]:
def experiment_three(X_train, y_train, X_test, y_test, top_indices):
    """
    On the same plot as 2, draw ROC curves and report the AUROC 
    values of logistic regression and Decision Trees on the 
    IMDB data binary classification task
    """
    
    models = [CustomLogisticRegression(learning_rate=0.01, epsilon=1e-5, max_iters=1e4), 
              DecisionTreeClassifier(), 
              KNeighborsClassifier(), 
              LR(C=1/0.01, fit_intercept=True, max_iter=int(1e4), tol=1e-5)] #sklearn logistic regression
    results = {}
    for model in models:
        model.fit(X_train, y_train)
        y_pred_prob_test = model.predict(X_test)
        y_pred_prob_train = model.predict(X_train)

        # Compute ROC curve and ROC area for the test set
        fpr_test, tpr_test, _ = roc_curve(y_test, y_pred_prob_test)
        roc_auc_test = auc(fpr_test, tpr_test)

        # Compute ROC curve and ROC area for the training set
        fpr_train, tpr_train, _ = roc_curve(y_train, y_pred_prob_train)
        roc_auc_train = auc(fpr_train, tpr_train)

        #results[f'{model.__class__.__name__} (train)'] = (fpr_train, tpr_train, roc_auc_train)
        results[f'{model.__class__.__name__} (test)'] = (fpr_test, tpr_test, roc_auc_test)
        
    plot_multiple_roc_curves(results,title='experiment_3_results.png')
    
#experiment_three(X_train_filtered, y_train_binary, X_test_filtered, y_test_binary, top_indices)

### Experiment 4 ###

In [ ]:
def experiment_four():
    """
    Report the:
    - Multiclass classification accuracy of multiclass linear regression 
    - Decision Trees on the 5 chosen classes from the 20-news-group data
    """
    pass

### Experiment 5 ###

In [ ]:
def experiment_five(X_train, y_train, X_test, y_test, top_indices):
    """
    Plot and compare the AUROC of DT and LogisticRegression as a function of the
    size of the dataset by controlling the training size.
    
    Randomly select 20%, 40%, 60%, and 80% of the available 
    training data and train your model on this subset and evaluate the trained 
    model on the held-out test set.
    """
    log_model = CustomLogisticRegression(learning_rate=0.01, epsilon=1e-5, max_iters=1e4)
    dt_model = DecisionTreeClassifier()
    results = {}  # Ex: '20%': (auroc_log, auroc_dt)
    
    for i in range(20, 101, 20):
        if i < 100:
            _, X_train_sub, _, y_train_sub = train_test_split(X_train, y_train, train_size=i/100, stratify=y_train)
        else:
            X_train_sub = X_train
            y_train_sub = y_train
        
        log_model.fit(X_train_sub, y_train_sub)
        dt_model.fit(X_train_sub, y_train_sub)
    
        y_pred_prob_test_log = log_model.predict(X_test)
        y_pred_prob_test_dt = dt_model.predict_proba(X_test)[:, 1]
        
        auroc_log = auc(*roc_curve(y_test, y_pred_prob_test_log)[:2])
        auroc_dt = auc(*roc_curve(y_test, y_pred_prob_test_dt)[:2])
        
        results[f'{i}%'] = (auroc_log, auroc_dt)
    
    # Plotting the results
    plot_imdb_data_auroc(results)
    
#experiment_five(X_train_filtered, y_train_binary, X_test_filtered, y_test_binary, top_indices)
    

### Experiment 6 ###

In [ ]:
def experiment_six(X_train, y_train, X_test, y_test):
    """
    Compare and evaluate the performance of different learning rates
    """
    histories = {}
    results = {}
    for i in range(1,5):
        lr = 10**-i
        model = CustomLogisticRegression(learning_rate=lr, epsilon=1e-5, max_iters=1e4, record_training=True)
        model.fit(X_train, y_train)
        y_pred_prob = model.predict(X_test)
        
        histories[str(lr)] = (model.loss_history, model.gradient_norm_history)
        fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
        roc_auc_train = auc(fpr, tpr)
        results[str(lr)] = (fpr, tpr, roc_auc_train)
        
    plot_model_convergence(histories, results)
        
#experiment_six(X_train_filtered, y_train_binary, X_test_filtered, y_test_binary)

### Experiment 7 ###

In [ ]:
def experiment_seven():
    """
    Evaluate the performance of the multiclass regression on more than 5 classes
    
    Compare the top k (e.g. k =3) predicted classes. 
    A correct prediction is determined by whether the true label is within the top k predicted labels. 
    The scoring mechanism involves assigning a score of 1 if the correct label is among the top k predictions and 0 otherwise.
    """
    pass